In [2]:
import copy

"""Construct CAS Hamiltonians with cropping
"""
import CAS_Cropping.saveload_utils as sl
import CAS_Cropping.ferm_utils as feru
import CAS_Cropping.csa_utils as csau
import CAS_Cropping.var_utils as varu
import openfermion as of
import numpy as np
from CAS_Cropping.sdstate import *
from itertools import product
import random
from pathlib import Path
import h5py
import sys
from CAS_Cropping.matrix_utils import construct_orthogonal
import pickle
import CAS.dmrghandler.src.dmrghandler.pyscf_wrappers
import CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import CAS.dmrghandler.src.dmrghandler as dmrghandler
import tensorflow as tf
import pyscf.tools.fcidump
import scipy
import pandas as pd
import os

### Parameters
tol = 1e-5
balance_strength = 2
save = False
# Number of spatial orbitals in a block
block_size = 7
# Number of electrons per block
ne_per_block = 4
# +- difference in number of electrons per block
ne_range = 0
# Number of killer operators for each CAS block
n_killer = 3
# Running Full CI to check compute the ground state, takes exponentially amount of time to execute
FCI = False
# Checking symmetries of the planted Hamiltonian, very costly
check_symmetry = False
# Concatenate the states in each block to compute the ground state solution
check_state = False

### Rename the fcidump files

We replace unsupported characters in file names to _

In [3]:
import os

In [4]:
def rename_files(directory):
    """
    Rename the files which have unsupported characters
    Args:
        directory: 

    Returns:

    """
    # List all files in the given directory
    files = os.listdir(directory)
    # Sort the files to ensure consistent order
    files.sort()
    
    for index, filename in enumerate(files):
        # Construct the old file path
        old_path = os.path.join(directory, filename)
        
        # Ensure we only rename files, not directories
        if os.path.isfile(old_path):
            # Replace slashes with underscores
            new_filename = filename.replace('/', '_')
            new_filename = new_filename.replace(',', '')
            
            new_filename = new_filename.replace(':', '_')
            
            new_filename = new_filename.replace('+', '_')
            
            new_filename = new_filename.replace('*', '_')
            
            # Remove all spaces
            new_filename = new_filename.replace(' ', '')
            
            # Construct the new file path
            new_path = os.path.join(directory, new_filename)
            
            try:
                # Rename the file
                os.rename(old_path, new_path)
            except OSError as e:
                print(f'Error renaming {old_path} to {new_path}: {e}')

In [5]:
rename_files(Path("../fcidumps_catalysts"))

FileNotFoundError: [Errno 2] No such file or directory: '../fcidumps_catalysts'

### Define necessary functions

In [6]:
def construct_blocks(size: int, total_orbs: int, spin_orb = False):
    """Construct CAS blocks with size for spin_orbs/spatial number of orbitals"""
    if spin_orb:
        size = size * 2
    blocks = []
    tmp = [0]
    
    # Here we form a list of length size until we reach 
    for i in range(1, total_orbs):
        if i % size == 0:
            blocks.append(tmp)
            tmp = [i]
        else:
            tmp.append(i)
    if len(tmp) != 0:
        blocks.append(tmp)
    return blocks

def construct_6_dominant_block(total_orbs: int):
    """Construct random CAS blocks """
    blocks = []
    first_block = [i for i in range(6)]
    blocks.append(first_block)
    tmp = [6]
    for i in range(7, total_orbs):
        if (i-6) % 3 == 0:
            blocks.append(tmp)
            tmp = [i]
        else:
            tmp.append(i)
    if len(tmp) != 0:
        blocks.append(tmp)
    return blocks


def construct_5_5_dominant_block(total_orbs: int):
    """Construct random CAS blocks """
    blocks = []
    first_block = [i for i in range(5)]
    second_block = [i+5 for i in range(5)]
    blocks.append(first_block)
    blocks.append(second_block)
    tmp = [10]
    for i in range(11, total_orbs):
        if (i-10) % 3 == 0:
            blocks.append(tmp)
            tmp = [i]
        else:
            tmp.append(i)
    if len(tmp) != 0:
        blocks.append(tmp)
    return blocks

def get_truncated_cas_tbt(H, blocks, casnum):
    """
    Trunctate the original Hamiltonian two body tensor into the cas block structures
    Args:
        H: hamiltonian in tuple
        blocks: The block partitioning
        casnum:

    Returns: cas_obt, cas_tbt, cas_x

    """
    Hobt, Htbt = H
    n = Htbt.shape[0]
    cas_tbt = np.zeros([n, n, n, n])
    cas_obt = np.zeros([n, n])
    cas_x = np.zeros(casnum)
    idx = 0
    for block in blocks:
        for p, q in product(block, repeat = 2):
            cas_obt[p,q] = Hobt[p,q]
            cas_x[idx] = Hobt[p,q]
            idx += 1
        for a, b, c, d in product(block, repeat = 4):
            cas_tbt [a,b,c,d] = Htbt [a,b,c,d]
            cas_x[idx] = Htbt[a,b,c,d]
            idx += 1
    return cas_obt, cas_tbt, cas_x

def solve_enums(H, blocks, e_total, ne_per_block = 0, ne_range = 0, balance_t = 50,):
    """Solve for number of electrons in each CAS block with FCI within the block,
    H = (obt, tbt) as the Hamiltonian in spatial orbitals.
    Notice that some quadratic terms (Ne-ne)^2 are added to ensure the correct number
    of electrons in the ground state of each block
    """
    cas_obt = H[0]
    cas_tbt = H[1]
    e_nums = []
    states = []
    E_cas = 0
    for index in range(len(blocks)):
        orbs = blocks[index]
        s = orbs[0]
        t = orbs[-1] + 1
        norbs = len(orbs)

        # Calculate the number of electrons in each CAS block
        if e_total - ne_per_block * (index + 1) >= 0:
            ne = ne_per_block
        elif e_total - ne_per_block * index >= 0:
            ne = e_total - ne_per_block * index
        else:
            ne = 0

        # Construct (Ne^-ne)^2 terms in matrix, to enforce structure of states 
        # for each CAS block in sd basis.
        if ne_per_block != 0:
            balance_obt = np.zeros([norbs, norbs])
            balance_tbt = np.zeros([norbs, norbs,  norbs, norbs])
            for p, q in product(range(norbs), repeat = 2):
                balance_tbt[p,p,q,q] += 1
            for p in range(len(orbs)):
                balance_obt[p,p] -= 2 * ne
        
        # Coefficient for (Ne^-ne)^2 term
        strength = balance_t * (1 + random.random())
    
        flag = True
        while flag:
            strength *= 2
            
            # Add the balance term to the truncated hamiltonian separately for one-body and two-body
            # The ne^2 term is a constant and doesn't affect the DMRG result so it's removed here.
            cas_tbt[s:t, s:t, s:t, s:t] = np.add(cas_tbt[s:t, s:t, s:t, s:t], strength * balance_tbt)
            cas_obt[s:t, s:t] = np.add(cas_obt[s:t, s:t], strength * balance_obt)
            # 
            # # Set spin_orb=False to represent spatial orbital basis Hamiltonian
            # # We find the FCI ground state energy
            ferm_op = feru.get_ferm_op(cas_tbt[s:t, s:t, s:t, s:t], spin_orb = False)
            ferm_op += feru.get_ferm_op(cas_obt[s:t, s:t], spin_orb = False)
            sparse_H_tmp = of.get_sparse_operator(ferm_op)
            tmp_E_min, t_sol = of.get_ground_state(sparse_H_tmp)
            st = sdstate(n_qubit = len(orbs) * 2)
            for i in range(len(t_sol)):
                if np.linalg.norm(t_sol[i]) > np.finfo(np.float32).eps:
                    st += sdstate(s = i, coeff = t_sol[i])
            st.normalize()
            E_st = st.exp(ferm_op)
            flag = False
        print(f"E_min: {tmp_E_min} for orbs: {orbs}")
        print(f"current state Energy: {E_st}")
        
        # The ground state energy of the whole hamiltonian is the sum of each 
        # CAS block minimum energy
        E_cas += E_st
        states.append(st)
        e_nums.append(ne)
    return e_nums, states, E_cas


def H_to_sparse(H: of.FermionOperator, n):
    """ Construct the sparse tensor representation of the Hamiltonian, represented by a constant term, a
    """
    h1e_keys = []
    h1e_vals = []
    h2e_keys = []
    h2e_vals = []
    for key, val in H.terms.items():
        if len(key) == 2:
            h1e_keys.append([key[0][0], key[1][0]])
            h1e_vals.append(val)
        elif len(key) == 4:
            h2e_keys.append([key[0][0], key[1][0], key[2][0], key[3][0]])
            h2e_vals.append(val)
    sparse_h1 = tf.sparse.SparseTensor(indices = h1e_keys, values = h1e_vals, dense_shape = [n,n])
    sparse_h2 = tf.sparse.SparseTensor(indices = h2e_keys, values = h2e_vals, dense_shape = [n,n,n,n])
    return sparse_h1, sparse_h2

def sparse_to_H(H_sparse):
    H = of.FermionOperator.zero()
    for _, (term, value) in enumerate(zip(H_sparse.indices, H_sparse.values)):
        index = [int(i) for i in term.numpy()]
        val = value.numpy()
        if len(index) == 2:
            H += of.FermionOperator(
                term = (
                    (index[0], 1), (index[1], 0)
                ),
                coefficient = val
            )
        elif len(index) == 4:
            H += of.FermionOperator(
                term = (
                    (index[0], 1), (index[1], 0),
                    (index[2], 1), (index[3], 0)
                ),
                coefficient = val
            )
    return H

def construct_killer(k, e_nums, n = 0, const = 1e-2, t = 1e2, n_killer = 3):
    """ Construct a killer operator for CAS Hamiltonian, based on cas block structure of k and the size of killer is
    given in k, the number of electrons in each CAS block of the ground state
    is specified by e_nums. t is the strength of quadratic balancing terms for the killer with respect to k,
    n_killer specifies the number of operators O to choose.
    """
    if not n:
        n = max([max(orbs) for orbs in k])
    killer = of.FermionOperator.zero()
    for i in range(len(k)):
        orbs = k[i]
        outside_orbs = [j for j in range(n) if j not in orbs]
        
        # Define Ne
        Ne = sum([of.FermionOperator("{}^ {}".format(i, i)) for i in orbs])
        
        # Construct O, for O as combination of Epq which preserves Sz and S2
        if len(outside_orbs) >= 4:
            tmp = 0
            while tmp < n_killer:
                p, q = random.sample(outside_orbs, 2)
                if abs(p - q) > 1:
                    
                    # Constructing symmetry conserved killers
                    O = of.FermionOperator.zero()
                    ferm_op = of.FermionOperator("{}^ {}".format(p, q)) + of.FermionOperator("{}^ {}".format(q, p))
                    O += ferm_op
                    O += of.hermitian_conjugated(ferm_op)
                    k_const = const * (1 + np.random.rand())
                    killer += k_const * O * (Ne - e_nums[i])
                    tmp += 1
        killer += t * (1 + np.random.rand()) * const * ((Ne - e_nums[i]) ** 2)
    killer_obt, killer_tbt = H_to_sparse(killer, n)
    # Killer constant term
    c = killer.terms[()]
    return c, killer_obt, killer_tbt

def get_param_num(n, k, complex = False):
    """
    Counting the parameters needed, where k is the number of orbitals occupied by CAS Fragments,
    and n-k orbitals are occupied by the CSA Fragments
    """
    if not complex:
        upnum = int(n * (n - 1) / 2)
    else:
        upnum = n * (n - 1)
    casnum = 0
    for block in k:
        casnum += len(block) ** 4 + len(block) ** 2
    pnum = upnum + casnum
    return upnum, casnum, pnum

In [7]:
def check_for_incorrect_spin_terms(tbt_to_check):
    num_incorrect_terms = 0
    #Check no incorrect spin terms present
    num_spin_orbitals = tbt_to_check.shape[0]
    no_incorrect_terms = True
    for piter in range(num_spin_orbitals):
        for qiter in range(num_spin_orbitals):
            for riter in range(num_spin_orbitals):
                for siter in range(num_spin_orbitals):
                    if piter % 2 == 0 and qiter % 2 == 0 and riter % 2 == 0 and siter % 2 == 0:
                        continue
                    if piter % 2 == 1 and qiter % 2 == 1 and riter % 2 == 1 and siter % 2 == 1:
                        continue
                    if piter % 2 == 0 and qiter % 2 == 0 and riter % 2 == 1 and siter % 2 == 1:
                        continue
                    if piter % 2 == 1 and qiter % 2 == 1 and riter % 2 == 0 and siter % 2 == 0:
                        continue
                    if not np.isclose(tbt_to_check[piter, qiter, riter, siter], 0.0):
                        # print(f"Incorrect spin term present in two body tensor at indices {piter}, {qiter}, {riter}, {siter}: {tbt_to_check[piter, qiter, riter, siter]}")
                        no_incorrect_terms = False
                        num_incorrect_terms += 1

    return no_incorrect_terms, num_incorrect_terms

def check_hamiltonian(obt, tbt_to_check):
    num_spin_orbitals = tbt_to_check.shape[0]
    no_incorrect_terms, num_incorrect_terms = check_for_incorrect_spin_terms(tbt_to_check)
    print(f"No incorrect spin terms present: {no_incorrect_terms}")
    print(f"Number of incorrect terms: {num_incorrect_terms}")

    spin_symm_check_passed = dmrghandler.dmrg_calc_prepare.check_spin_symmetry(one_body_tensor=obt, two_body_tensor=tbt_to_check)
    print(f"Spin symmetry check passed: {spin_symm_check_passed}")

    permutation_symmetries_complex_orbitals_check_passed = dmrghandler.dmrg_calc_prepare.check_permutation_symmetries_complex_orbitals(
        obt, tbt_to_check
    )
    print(f"Permutation symmetries complex orbitals check passed: {permutation_symmetries_complex_orbitals_check_passed}")

    permutation_symmetries_real_orbitals_check_passed = dmrghandler.dmrg_calc_prepare.check_permutation_symmetries_real_orbitals(
        obt, tbt_to_check
    )
    print(f"Permutation symmetries real orbitals check passed: {permutation_symmetries_real_orbitals_check_passed}")

def chem_spatial_orb_to_phys_spatial_orb(obt, tbt):
    """
    Converts the spatial orbital chemist notation into physcicist notation
    Args:
        obt:
        tbt:

    Returns:

    """
    phy_obt = obt + np.einsum("prrq->pq", tbt)
    phy_tbt = 2 * tbt
    return phy_obt, phy_tbt

def get_cas_matrix(cas_x, n, k):
    obt = np.zeros([n, n])
    tbt = np.zeros([n, n, n, n])
    idx = 0
    for orbs in k:
        for p, q in product(orbs, repeat = 2):
            obt[p,q] = cas_x[idx]
            idx += 1
        for p, q, r, s in product(orbs, repeat = 4):
            tbt[p,q,r,s] = cas_x[idx]
            idx += 1
    return obt, tbt

def orbtransf(tensor, U, complex = False):
    """Return applying UHU* for the tensor representing the 1e or 2e tensor"""
    if len(tensor.shape) == 4:
        p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor)[0]
        return np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor, optimize = p)
    elif len(tensor.shape) == 2:
        p = np.einsum_path('ap,bq, pq->ab', U, U, tensor)[0]
        return np.einsum('ap,bq, pq->ab', U, U, tensor, optimize = p)
def load_Hamiltonian_with_solution(obt, tbt, k_obt, k_tbt, E_min):
    """Load the precomputed Hamiltonian with the given file_name. Return the Hamiltonians and the state
    Hamiltonian is represented in spatial orbital basis as three terms: (c, h_pq, g_pqrs)
    Returns:
    U: Arbitrary Unitary Rotation, in spatial orbital basis
    H_cas: Unhidden CAS Fragements
    H_hidden：U H_cas U*
    H_with_killer: H_cas + killer
    H_killer_hidden: U H_with_killer U*
    """
    # with open(path + file_name, 'rb') as handle:
    #     dic = pickle.load(handle)
    #     key = list(dic.keys())[0]
    #     dic = dic[key]
    # E_min = dic["E_min"]
    # cas_x = dic["cas_x"]
    # killer = dic["killer"]
    # killer_c = killer[0]
    # sol = dic["sol"]
    k_obt = tf.sparse.reorder(k_obt)
    k_tbt = tf.sparse.reorder(k_tbt)
    # k = dic["k"]
    # upnum = dic["upnum"]
    # spatial_orbs = dic["spatial_orbs"]
    # obt = dic["cas_obt"]
    # tbt = dic["cas_tbt"]
    # 
    # print(f"in load tbt check {tbt[0, 0, 0, 0]}")

    # CAS 2e tensor
    # obt, tbt = get_cas_matrix(cas_x, spatial_orbs, k)
    H_cas = (0, obt, tbt)
    H_with_killer = (killer_c, obt+tf.sparse.to_dense(k_obt), tbt+tf.sparse.to_dense(k_tbt))
    
    # Set up random unitary to hide 2e tensor
    random_uparams = np.random.rand(upnum) * 0.01
    U = construct_orthogonal(spatial_orbs, random_uparams)
    # Hide 2e etensor with random unitary transformation
    H_hidden = (0, orbtransf(obt, U), orbtransf(tbt, U))
    H_killer_hidden = (killer_c, orbtransf(H_with_killer[1], U), orbtransf(H_with_killer[2], U))
    return U, H_cas, H_hidden, H_with_killer, H_killer_hidden, E_min

In [8]:
def load_Hamiltonian_with_solution_diag_U(U_diag, obt, tbt, k_obt, k_tbt, E_min):
    """Load the precomputed Hamiltonian with the given file_name. Return the Hamiltonians and the state
    Hamiltonian is represented in spatial orbital basis as three terms: (c, h_pq, g_pqrs)
    Returns:
    U: Arbitrary Unitary Rotation, in spatial orbital basis
    H_cas: Unhidden CAS Fragements
    H_hidden：U H_cas U*
    H_with_killer: H_cas + killer
    H_killer_hidden: U H_with_killer U*
    """
    # with open(path + file_name, 'rb') as handle:
    #     dic = pickle.load(handle)
    #     key = list(dic.keys())[0]
    #     dic = dic[key]
    # E_min = dic["E_min"]
    # cas_x = dic["cas_x"]
    # killer = dic["killer"]
    # killer_c = killer[0]
    # sol = dic["sol"]
    # k_obt = tf.sparse.reorder(killer[1])
    # k_tbt = tf.sparse.reorder(killer[2])
    # k = dic["k"]
    # upnum = dic["upnum"]
    # spatial_orbs = dic["spatial_orbs"]
    # obt = dic["cas_obt"]
    # tbt = dic["cas_tbt"]
    
    k_obt = tf.sparse.reorder(k_obt)
    k_tbt = tf.sparse.reorder(k_tbt)
    
    print(f"in load tbt check {tbt[0, 0, 0, 0]}")

    # CAS 2e tensor
    # obt, tbt = get_cas_matrix(cas_x, spatial_orbs, k)
    H_cas = (0, obt, tbt)
    H_with_killer = (killer_c, obt+tf.sparse.to_dense(k_obt), tbt+tf.sparse.to_dense(k_tbt))

    # Hide 2e etensor with random unitary transformation
    H_hidden = (0, orbtransf(obt, U_diag), orbtransf(tbt, U_diag))
    H_killer_hidden = (killer_c, orbtransf(H_with_killer[1], U_diag), orbtransf(H_with_killer[2], U_diag))
    return U_diag, H_cas, H_hidden, H_with_killer, H_killer_hidden, E_min

In [9]:
def decrease_diagonal(tensor):
    n = tensor.shape[0]
    if tensor.shape == (n, n):
        one_body = tensor.copy()
        for i in range(n):
            one_body[i, i] *= 1
        return one_body
    if tensor.shape == (n, n, n, n):
        two_body = tensor.copy()
        for p in range(n):
            for q in range(n):
                two_body[p, p, q, q] *= 1
                two_body[p, q, q, p] *= 1
                two_body[p, q, p, q] *= 1
        return two_body

### We iteratively generate FCIDUMP files
We iteratively generate FCIDUMP files from the catalysts in the given directory. We also save the data to an excel sheet.


In [10]:
writer = pd.ExcelWriter("fci_data.xlsx", engine='xlsxwriter')
fcidumps = []
e_mins = []
e_min_killers = [] 

In [11]:
fcidump_path = Path("fcidumps_LPBLISS_small_test")
fcidump_output_path = Path("block_size_test")

In [13]:
for fcidump_filename in os.listdir(fcidump_path):
    (
    one_body_tensor,
    two_body_tensor,
    nuc_rep_energy,
    num_orbitals,
    num_spin_orbitals,
    num_electrons,
    two_S,
    two_Sz,
    orb_sym,
    extra_attributes,
    ) = dmrghandler.dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=Path(fcidump_path)/Path(fcidump_filename), molpro_orbsym_convention=True)
    print(two_body_tensor)
    spin_orbs = 2*num_orbitals
    spatial_orbs = num_orbitals
    print(f"Nuc energy: {nuc_rep_energy}")
    print(one_body_tensor)
    
    spin_orbs = 2*num_orbitals
    spatial_orbs = num_orbitals
    
    print(f"Number of spin orbitals: {spin_orbs}")
    
    Hobt = one_body_tensor
    Htbt = two_body_tensor
    Hobt -= 0.5 * np.einsum("prrq->pq", Htbt.copy())
    Htbt *= 0.5
    
    # Hobt = decrease_diagonal(Hobt * 10)
    # Htbt = decrease_diagonal(Htbt * 10)
    
    ### Uncomment if we need to use U_diag ###
    # D, U = np.linalg.eigh(Hobt)
    # U_diag = U.T
    # Hobt = orbtransf(Hobt, U_diag)
    # Htbt = orbtransf(Htbt, U_diag)
    
    
    
    H = (Hobt, Htbt)
    print(f"Htbt check {Htbt[0, 0, 0, 0]}")
    k = construct_blocks(block_size, spatial_orbs)
    print(k)
    #k = construct_5_5_dominant_block(spatial_orbs)
    # k = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11, 12, 13], [14, 15, 16], [17, 18, 19], [20, 21, 22, 23, 24], [25, 26, 27], [28, 29, 30], [31]]
    k = [[0, 1, 2], [3, 4], [5, 6]]
    upnum, casnum, pnum = get_param_num(spatial_orbs, k, complex=False)
    cas_obt, cas_tbt, cas_x = get_truncated_cas_tbt(H, k, casnum)
    
    print("Block constructed")
    
    planted_sol = {}
    H_cas = [cas_obt, cas_tbt]
    e_nums, states, E_cas = solve_enums(H_cas, k, num_electrons, ne_per_block = ne_per_block,
                                        ne_range = ne_range, balance_t = balance_strength)
    print(f"After solve check ${H_cas[1][0, 0, 0, 0]}")
    sd_sol = sdstate()
    obt_2 = copy.deepcopy(cas_obt)
    tbt_2 = copy.deepcopy(cas_tbt)
    
    print("Solve Enums")
    
    killer_c, killer_obt, killer_tbt = construct_killer(k, e_nums, n = spatial_orbs, n_killer = n_killer)
    
    ### Uncomment the following lines to add customized Unitaries ###
    # n = num_orbitals
    # permutation = np.random.permutation(n)
    # identity_matrix = np.eye(n)
    # U_perm = identity_matrix[permutation]
    
    # Add noise 
    # taking the log of the matrix will introduce error.
    # M_gen = np.real(scipy.linalg.logm(U_perm))
    # n = len(M_gen[0])
    
    # Make sure M_gen is anti symmetric
    # for i in range(n):
    #     M_gen[i, i] = 0
    # 
    # A = np.random.rand(n, n)
    
    # Make the matrix anti-symmetric
    # M_antisymmetric = A - A.T
    # M_new = M_gen + M_antisymmetric * 0
    # 
    # for p, q in product(range(n), range(n)):
    #     assert np.isclose(M_gen[p, q], - M_gen[q, p])
    # 
    # for n in range(len(M_new[0])):
    #     M_new[n, n] = 0
        
    U, H_cas, H_hidden, H_with_killer, H_killer_hidden, E_min = load_Hamiltonian_with_solution(cas_obt, cas_tbt, killer_obt, killer_tbt, E_cas)
    print(f"E_min: {E_min}")
    fcidumps.append(fcidump_filename)
    e_mins.append(E_min)
    e_min_killers.append(E_min - H_with_killer[0])
    
    print(f"Htbt check again {H_cas[2][0, 0, 0, 0]}")
    
    tbt_1_H_ij, tbt_1_G_ijkl = chem_spatial_orb_to_phys_spatial_orb(H_cas[1], H_cas[2])
    tbt_1_hidden_H_ij, tbt_1_hidden_G_ijkl = chem_spatial_orb_to_phys_spatial_orb(H_hidden[1], H_hidden[2])
    tbt_1_hidden_H_ij = np.float64(np.real(tbt_1_hidden_H_ij))
    tbt_1_hidden_G_ijkl = np.float64(np.real(tbt_1_hidden_G_ijkl))
    
    L2_orig = scipy.linalg.norm(one_body_tensor, ord=None) - scipy.linalg.norm(tbt_1_hidden_H_ij, ord=None)
    L2_orig2 = scipy.linalg.norm(two_body_tensor, ord=None) - scipy.linalg.norm(tbt_1_hidden_G_ijkl, ord=None)
    print(f"diff orig and rotated: {L2_orig}, {L2_orig2}")
    
    # Taking the L2-norm difference between the non-hiding and hiding tensors 
    # Making sure they are close
    L2_1 =  scipy.linalg.norm(tbt_1_H_ij, ord=None) - scipy.linalg.norm(tbt_1_hidden_H_ij, ord=None)
    L2_2 = scipy.linalg.norm(tbt_1_G_ijkl, ord=None) - scipy.linalg.norm(tbt_1_hidden_G_ijkl, ord=None)
    print(L2_1, L2_2)
    print(scipy.linalg.norm(tbt_1_G_ijkl - tbt_1_hidden_G_ijkl, ord=None))
    
    tbt_3_H_ij, tbt_3_G_ijkl = chem_spatial_orb_to_phys_spatial_orb(H_with_killer[1], H_with_killer[2])
    tbt_3_H_ij = np.float64(np.real(tbt_3_H_ij))
    tbt_3_G_ijkl = np.float64(np.real(tbt_3_G_ijkl))
    
    tbt_3_hidden_H_ij, tbt_3_hidden_G_ijkl = chem_spatial_orb_to_phys_spatial_orb(H_killer_hidden[1], H_killer_hidden[2])
    tbt_3_hidden_H_ij = np.float64(np.real(tbt_3_hidden_H_ij))
    tbt_3_hidden_G_ijkl = np.float64(np.real(tbt_3_hidden_G_ijkl))
    
    # Make dir
    fcidump_output_path = Path(fcidump_output_path)
    fcidump_output_path.mkdir(parents=True, exist_ok=True)
    
    print(tbt_1_H_ij)
    
    label = "_tbt_1_block_test"
    # label = "_tbt_1"
    filename = fcidump_filename + f"{label}"
    pyscf.tools.fcidump.from_integrals(
        fcidump_output_path/Path(filename),
        tbt_1_H_ij,
        tbt_1_G_ijkl,
        nmo=tbt_1_G_ijkl.shape[0],
        nelec=np.sum(e_nums),
        nuc=nuc_rep_energy,
        ms=two_S,
        # ms=1,
        orbsym=None,
        tol=1E-8,
        float_format=' %.16g',
    )

    # label = "_tbt_1_hidden"
    # # label = "_tbt_1_hidden"
    # filename = fcidump_filename + f"{label}"
    # pyscf.tools.fcidump.from_integrals(
    #     fcidump_output_path/Path(filename),
    #     tbt_1_hidden_H_ij,
    #     tbt_1_hidden_G_ijkl,
    #     nmo=tbt_1_G_ijkl.shape[0],
    #     nelec=np.sum(e_nums),
    #     nuc=nuc_rep_energy,
    #     ms=two_S,
    #     # ms=1,
    #     orbsym=None,
    #     tol=1E-8,
    #     float_format=' %.16g',
    # )
    # label = "_tbt_3_enum_skew"
    # # label = "_tbt_3"
    # filename = fcidump_filename + f"{label}"
    # pyscf.tools.fcidump.from_integrals(
    #     fcidump_output_path/Path(filename),
    #     tbt_3_H_ij,
    #     tbt_3_G_ijkl,
    #     nmo=tbt_3_H_ij.shape[0],
    #     nelec=np.sum(e_nums),
    #     nuc=nuc_rep_energy,
    #     ms=two_S,
    #     orbsym=None,
    #     tol=1E-8,
    #     float_format=' %.16g',
    # )
    # label = "_tbt_3_hidden_size51"
    # # label = "_tbt_3_hidden"
    # filename = fcidump_filename + f"{label}"
    # pyscf.tools.fcidump.from_integrals(
    #     fcidump_output_path/Path(filename),
    #     tbt_3_hidden_H_ij,
    #     tbt_3_hidden_G_ijkl,
    #     nmo=tbt_3_hidden_H_ij.shape[0],
    #     nelec=np.sum(e_nums),
    #     nuc=nuc_rep_energy,
    #     ms=two_S,
    #     orbsym=None,
    #     tol=1E-8,
    #     float_format=' %.16g',
    # )

Parsing fcidumps_LPBLISS_small_test/fcidump.36_1ru_II_2pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}
[[[[ 6.47958225e-01 -1.48187031e-03  1.96960690e-04 ... -7.03171600e-06
    -1.94340466e-02  1.28360828e-02]
   [-1.48187031e-03  5.78836058e-01  9.36623232e-04 ... -1.19758635e-03
     4.83105397e-03  5.20727177e-03]
   [ 1.96960690e-04  9.36623232e-04  6.06129879e-01 ... -1.01883103e-02
     2.89368232e-03 -5.11325572e-03]
   ...
   [-7.03171600e-06 -1.19758635e-03 -1.01883103e-02 ...  5.97363895e-01
     1.04289012e-03 -9.83688926e-04]
   [-1.94340466e-02  4.83105397e-03  2.89368232e-03 ...  1.04289012e-03
     3.50241460e-01 -3.40382990e-03]
   [ 1.28360828e-02  5.20727177e-03 -5.11325572e-03 ... -9.83688926e-04
    -3.40382990e-03  3.05440938e-01]]

  [[-1.48187031e-03  3.20773756e-02 -7.07323579e-04 ...  4.83333365e-04
     7.46647432e-03 -1.78339437e-03]
   [ 3.20773756e-02 -1.53436874e-03 -1.98123668e-04 ... -2.75022487e-04
     1.55441449e-02 -3.65698545e-03]
   [-7.07323579e-04 -1

In [15]:
print(e_mins)
print(e_min_killers)

[-1721.6043145966973, -1470.5090778231288, -824.899081919752, -2424.064932870167, -2131.4740926959007, -792.4717519224702]
[-2029.7855588643754, -1756.5153677195008, -1000.2652889042487, -2866.5055658595566, -2461.455669174869, -958.9975163543494]


### Get the list of filenames

In [66]:
files = sorted(os.listdir(Path("fcidump_spin_test")))
for fcidump_filename in files:
    print("\""+fcidump_filename+"\",")

print(fcidump_filename[8:14] + fcidump_filename[-17:-6])

".DS_Store",
"fcidump.32_2ru_III_3pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}_tbt_1_enum",
32_2rul2tz'}_tbt_


In [67]:
files = sorted(os.listdir(Path("fcidump_spin_test")))
for fcidump_filename in files:
    print("\""+ fcidump_filename[8:14] + fcidump_filename[-17:-3] + "\",")

"e.DS_St",
"32_2rul2tz'}_tbt_1_e",


In [20]:
df_1 = pd.DataFrame()
column_names = ["FCIDUMP","E_min", "E_min_killer"]
data = [fcidumps, e_mins, e_min_killers]
#Iteratively add data to new columns
for column_name, column_data in zip(column_names, data):
    df_1[column_name] = column_data

# Specify the path where you want to save the CSV file
csv_path = f"e_min_orb_reorder.csv"

# Save DataFrame to a CSV file
df_1.to_csv(csv_path, index=False)

### Apply basis transformation to diagonalize h1e
Then, we use the same unitary to rotate h2e. We then use the same unitary to hide the CAS block structure

### Small change on diagonalizing unitary

In [587]:
# M_gen = scipy.linalg.logm(U_diag)
# n = len(M_gen[0])
# A = np.random.rand(n, n) + 1j * np.random.rand(n, n)
# M_rand = (A + A.T.conj()) / 10
# M_new = M_gen + M_rand

### Permutation unitary 

In [598]:
label = "_tbt_1_perm_noise_1000"
# label = "_tbt_1"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_1_H_ij,
    tbt_1_G_ijkl,
    nmo=tbt_1_G_ijkl.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=two_S,
    # ms=1,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [599]:
label = "_tbt_1_hidden_perm_noise_1000"
# label = "_tbt_1_hidden"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_1_hidden_H_ij,
    tbt_1_hidden_G_ijkl,
    nmo=tbt_1_G_ijkl.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=two_S,
    # ms=1,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [600]:
label = "_tbt_3_perm_noise_1000"
# label = "_tbt_3"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_3_H_ij,
    tbt_3_G_ijkl,
    nmo=tbt_3_H_ij.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=two_S,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [601]:
label = "_tbt_3_hidden_perm_noise_1000"
# label = "_tbt_3_hidden"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_3_hidden_H_ij,
    tbt_3_hidden_G_ijkl,
    nmo=tbt_3_hidden_H_ij.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=two_S,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [535]:
U = scipy.linalg.expm([[0, 0.02, 0, 0.01], [-0.02, 0, 0.03, 0], [0, -0.03, 0, 0.05], [-0.01, 0,-0.05, 0]])
print(U)
print(np.matmul(U.T, U))

[[ 9.99750011e-01  1.99978335e-02  4.99837519e-05  9.99999986e-03]
 [-1.99978335e-02  9.99350141e-01  2.99826696e-02  6.49788774e-04]
 [ 4.99837519e-05 -2.99826696e-02  9.98300482e-01  4.99718381e-02]
 [-9.99999986e-03  6.49788774e-04 -4.99718381e-02  9.98700352e-01]]
[[ 1.00000000e+00 -2.69261437e-18 -1.25477388e-20  3.98852660e-18]
 [-2.69261437e-18  1.00000000e+00 -5.78287750e-18 -2.11635713e-19]
 [-1.25477388e-20 -5.78287750e-18  1.00000000e+00  6.66828388e-18]
 [ 3.98852660e-18 -2.11635713e-19  6.66828388e-18  1.00000000e+00]]


In [190]:
n = 6
random_uparams = np.random.rand(n * (n - 1)) * 0.01
U = construct_orthogonal(n, np.array([1 for _ in range((n*(n-1)))]))
print(U)

[[ 0.22464417 -0.28634654 -0.39497455 -0.37781943 -0.07931641  0.7499981 ]
 [-0.7499981   0.22464417 -0.28634654 -0.39497455 -0.37781943 -0.07931641]
 [ 0.07931641 -0.7499981   0.22464417 -0.28634654 -0.39497455 -0.37781943]
 [ 0.37781943  0.07931641 -0.7499981   0.22464417 -0.28634654 -0.39497455]
 [ 0.39497455  0.37781943  0.07931641 -0.7499981   0.22464417 -0.28634654]
 [ 0.28634654  0.39497455  0.37781943  0.07931641 -0.7499981   0.22464417]]


In [191]:
np.random.rand(n * (n - 1)) * 0.01


array([0.00040318, 0.00597534, 0.00854188, 0.00548401, 0.00273792,
       0.00590013, 0.00401979, 0.00524673, 0.00345726, 0.00095822,
       0.00513977, 0.00598424, 0.00500388, 0.00325614, 0.00309336,
       0.0066578 , 0.00839007, 0.00717317, 0.0019754 , 0.00054215,
       0.00970255, 0.00743275, 0.00266306, 0.00861869, 0.00185032,
       0.0084918 , 0.00927782, 0.00252384, 0.00249669, 0.00926987])

In [192]:
[[ 0.76652907  0.20343622  0.20827651  0.07401923  0.54170241  0.16955073]
 [-0.41055064  0.72035932  0.10143876  0.31426899  0.09012948  0.44198696]
 [-0.35213684 -0.39868884  0.77642264 -0.02605027  0.32261458  0.09724172]
 [-0.09407637 -0.4272814  -0.37571675  0.75165779  0.30444458  0.09869771]
 [-0.31928106  0.03475548 -0.42229018 -0.50238398  0.68167586 -0.03809223]
 [ 0.09534278 -0.31139368 -0.15494156 -0.27874441 -0.19231883  0.86905228]]


SyntaxError: invalid syntax (2313808475.py, line 1)